This project creates a network that can generate text. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
path_to_file = '/content/drive/MyDrive/FINAL_TF2_FILES/TF_2_Notebooks_and_Data/06-NLP-and-Text-Data/shakespeare.txt'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
text = open(path_to_file, 'r').read()

In [5]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print(vocab)
len(vocab)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '}']


84

**Text Processing**

A neural network can't take in the raw string data, so we need to assign numbers to each character. This will create two dictionaries that can go from numeric index to character and character to numeric index.

In [7]:
char_to_ind = {u:i for i, u in enumerate(vocab)}

In [8]:
char_to_ind

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '<': 23,
 '>': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51,
 '[': 52,
 ']': 53,
 '_': 54,
 '`': 55,
 'a': 56,
 'b': 57,
 'c': 58,
 'd': 59,
 'e': 60,
 'f': 61,
 'g': 62,
 'h': 63,
 'i': 64,
 'j': 65,
 'k': 66,
 'l': 67,
 'm': 68,
 'n': 69,
 'o': 70,
 'p': 71,
 'q': 72,
 'r': 73,
 's': 74,
 't': 75,
 'u': 76,
 'v': 77,
 'w': 78,
 'x': 79,
 'y': 80,
 'z': 81,
 '|': 82,
 '}': 83}

In [9]:
ind_to_char = np.array(vocab)

In [22]:
ind_to_char

array(['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',
       'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
       'w', 'x', 'y', 'z', '|', '}'], dtype='<U1')

In [23]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [24]:
encoded_text

array([ 0,  1,  1, ..., 30, 39, 29])

In [25]:
sample = text[:20]
sample

'\n                   '

In [26]:
encoded_text[:20]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

**Creating Batches**

In [14]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [15]:
line = "From fairest creatures we desire increase"

In [16]:
len(line)

41

In [17]:
part_stanza = """From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,"""

In [18]:
len(part_stanza)

131

In [19]:
seq_len = 120

In [20]:
total_num_seq = len(text)//(seq_len+1)

In [21]:
total_num_seq

45005

In [27]:
# Create Training Sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

for i in char_dataset.take(500):
     print(ind_to_char[i.numpy()])



 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1


 
 
F
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
e
s
 
w
e
 
d
e
s
i
r
e
 
i
n
c
r
e
a
s
e
,


 
 
T
h
a
t
 
t
h
e
r
e
b
y
 
b
e
a
u
t
y
'
s
 
r
o
s
e
 
m
i
g
h
t
 
n
e
v
e
r
 
d
i
e
,


 
 
B
u
t
 
a
s
 
t
h
e
 
r
i
p
e
r
 
s
h
o
u
l
d
 
b
y
 
t
i
m
e
 
d
e
c
e
a
s
e
,


 
 
H
i
s
 
t
e
n
d
e
r
 
h
e
i
r
 
m
i
g
h
t
 
b
e
a
r
 
h
i
s
 
m
e
m
o
r
y
:


 
 
B
u
t
 
t
h
o
u
 
c
o
n
t
r
a
c
t
e
d
 
t
o
 
t
h
i
n
e
 
o
w
n
 
b
r
i
g
h
t
 
e
y
e
s
,


 
 
F
e
e
d
'
s
t
 
t
h
y
 
l
i
g
h
t
'
s
 
f
l
a
m
e
 
w
i
t
h
 
s
e
l
f
-
s
u
b
s
t
a
n
t
i
a
l
 
f
u
e
l
,


 
 
M
a
k
i
n
g
 
a
 
f
a
m
i
n
e
 
w
h
e
r
e
 
a
b
u
n
d
a
n
c
e
 
l
i
e
s
,


 
 
T
h
y
 
s
e
l
f
 
t
h
y
 
f
o
e
,
 
t
o
 
t
h
y
 
s
w
e
e
t
 
s
e
l
f
 
t
o
o
 
c
r
u
e
l
:


 
 
T
h
o
u
 
t
h
a
t
 
a
r
t
 
n
o
w
 
t
h
e
 
w
o
r
l
d
'
s
 
f
r
e
s
h
 
o
r
n
a
m
e
n
t
,


 
 
A
n
d
 
o
n
l
y
 
h
e
r
a
l
d
 
t
o
 
t
h
e
 
g
a
u
d
y
 
s
p
r
i
n
g
,


 
 
W
i
t
h
i
n
 
t
h
i
n
e
 
o
w
n
 
b
u


In [28]:
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

In [29]:
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt

In [30]:
dataset = sequences.map(create_seq_targets)

In [31]:
for input_txt, target_txt in  dataset.take(1):
    print(input_txt.numpy())
    print(''.join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    
    print(''.join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


In [32]:
# Batch size
batch_size = 128

# Buffer size to shuffle the dataset so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in which it shuffles elements
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [33]:
dataset

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int64, tf.int64)>

**Step 4: Creating the Model**

In [34]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embed_dim = 64

# Number of RNN units
rnn_neurons = 1026

In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout,GRU

In [36]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [37]:
def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [38]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim,batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

In [39]:
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)

In [40]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5376      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 84)           86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


**Step 5: Training the model**

In [41]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Predict off some random batch
  example_batch_predictions = model(input_example_batch)

  # Display the dimensions of the predictions
  print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")

(128, 120, 84)  <=== (batch_size, sequence_length, vocab_size)


In [42]:
example_batch_predictions

<tf.Tensor: shape=(128, 120, 84), dtype=float32, numpy=
array([[[ 7.05354940e-03,  1.69501815e-03,  3.49339982e-03, ...,
          1.77800853e-03, -2.90400372e-03, -7.05008861e-03],
        [ 3.79843800e-03,  3.95623688e-03,  3.32556479e-03, ...,
         -6.14337670e-03,  1.36622880e-03, -7.75758270e-03],
        [ 4.58277576e-03,  1.31724612e-03,  9.54426639e-03, ...,
         -1.68735324e-03,  8.08260962e-03, -3.24574625e-03],
        ...,
        [ 6.98074978e-03, -2.67161592e-03,  7.38231977e-03, ...,
          4.13243333e-03,  1.06335888e-02,  1.88344915e-03],
        [ 1.22279243e-03, -4.13206127e-03,  2.09051277e-03, ...,
          9.95035376e-03,  5.05823735e-03, -1.51434424e-03],
        [ 1.00577355e-03,  2.62995134e-03,  1.38334453e-03, ...,
         -9.46962449e-04,  5.38224494e-03, -5.79822715e-03]],

       [[ 5.52075217e-03, -4.13775584e-03,  3.56247555e-03, ...,
          3.28827789e-03,  6.61047176e-03,  3.10903671e-03],
        [ 4.31171153e-04, -5.28571242e-03, -5.3

In [43]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [44]:
sampled_indices

<tf.Tensor: shape=(120, 1), dtype=int64, numpy=
array([[78],
       [27],
       [68],
       [56],
       [ 3],
       [ 6],
       [81],
       [30],
       [77],
       [28],
       [80],
       [44],
       [65],
       [12],
       [52],
       [65],
       [63],
       [48],
       [21],
       [27],
       [ 7],
       [63],
       [82],
       [40],
       [63],
       [10],
       [ 9],
       [ 9],
       [73],
       [37],
       [35],
       [22],
       [13],
       [68],
       [73],
       [ 8],
       [15],
       [41],
       [47],
       [18],
       [82],
       [71],
       [39],
       [74],
       [19],
       [75],
       [29],
       [83],
       [80],
       [54],
       [80],
       [74],
       [52],
       [23],
       [62],
       [72],
       [23],
       [45],
       [54],
       [ 9],
       [61],
       [68],
       [37],
       [40],
       [52],
       [80],
       [67],
       [62],
       [30],
       [78],
       [50],
       [72],
       [30],
   

In [45]:
# Reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [46]:
sampled_indices

array([78, 27, 68, 56,  3,  6, 81, 30, 77, 28, 80, 44, 65, 12, 52, 65, 63,
       48, 21, 27,  7, 63, 82, 40, 63, 10,  9,  9, 73, 37, 35, 22, 13, 68,
       73,  8, 15, 41, 47, 18, 82, 71, 39, 74, 19, 75, 29, 83, 80, 54, 80,
       74, 52, 23, 62, 72, 23, 45, 54,  9, 61, 68, 37, 40, 52, 80, 67, 62,
       30, 78, 50, 72, 30, 12, 37, 37, 83, 40, 64, 46, 41, 16, 76, 35, 80,
       50, 65, 33, 44, 26, 12, 80, 49, 48, 25, 11, 14, 43,  9, 37, 73, 51,
        6, 53, 31, 14,  3, 36, 60, 83, 64, 10, 14, 44,  5, 21, 21,  1, 58,
        7])

In [47]:
print("Given the input seq: \n")
print("".join(ind_to_char[input_example_batch[0]]))
print('\n')
print("Next Char Predictions: \n")
print("".join(ind_to_char[sampled_indices ]))

Given the input seq: 

s here contain'd relish of love,
    Of my lord's health, of his content; yet not
    That we two are asunder- let that 


Next Char Predictions: 

wBma"(zEvCySj1[jhW:B)h|Oh.--rLJ;2mr,4PV7|pNs8tD}y_ys[<gq<T_-fmLO[ylgEwYqE1LL}OiUP5uJyYjHSA1yXW?03R-LrZ(]F3"Ke}i.3S':: c)


In [48]:
epochs = 30

In [ ]:
model.fit(dataset,epochs=epochs)

Epoch 1/30
351/351 [==============================] - 53s 143ms/step - loss: 2.5368
Epoch 2/30
351/351 [==============================] - 49s 138ms/step - loss: 1.7139
Epoch 3/30
351/351 [==============================] - 52s 144ms/step - loss: 1.4497
Epoch 4/30
351/351 [==============================] - 51s 142ms/step - loss: 1.3345
Epoch 5/30
351/351 [==============================] - 49s 138ms/step - loss: 1.2736
Epoch 6/30
351/351 [==============================] - 51s 141ms/step - loss: 1.2343
Epoch 7/30
351/351 [==============================] - 50s 139ms/step - loss: 1.2048
Epoch 8/30
351/351 [==============================] - 52s 145ms/step - loss: 1.1815
Epoch 9/30
351/351 [==============================] - 49s 138ms/step - loss: 1.1614
Epoch 10/30
351/351 [==============================] - 52s 144ms/step - loss: 1.1443
Epoch 11/30
351/351 [==============================] - 49s 136ms/step - loss: 1.1289
Epoch 12/30
351/351 [==============================] - 51s 141ms/step - lo

**Step 6: Generating text**

In [ ]:
model.save('shakespeare2_gen.h5') 

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

model.load_weights('shakespeare2_gen.h5')

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 64)             5376      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1026)           3361176   
_________________________________________________________________
dense_1 (Dense)              (1, None, 84)             86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
  '''
  model: Trained Model to Generate Text
  start_seed: Intial Seed text in string form
  gen_size: Number of characters to generate

  Basic idea behind this function is to take in some seed text, format it so
  that it is in the correct shape for our network, then loop the sequence as
  we keep adding our own predicted characters. Similar to our work in the RNN
  time series problems.
  '''

  # Number of characters to generate
  num_generate = gen_size

  # Vecotrizing starting seed text
  input_eval = [char_to_ind[s] for s in start_seed]

  # Expand to match batch format shape
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty list to hold resulting generated text
  text_generated = []

  # Temperature effects randomness in our resulting text
  # The term is derived from entropy/thermodynamics.
  # The temperature is used to effect probability of next characters.
  # Higher probability == lesss surprising/ more expected
  # Lower temperature == more surprising / less expected
 
  temperature = temp

  # Here batch size == 1
  model.reset_states()

  for i in range(num_generate):

      # Generate Predictions
      predictions = model(input_eval)

      # Remove the batch shape dimension
      predictions = tf.squeeze(predictions, 0)

      # Use a cateogircal disitribution to select the next character
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted charracter for the next input
      input_eval = tf.expand_dims([predicted_id], 0)

      # Transform back to character letter
      text_generated.append(ind_to_char[predicted_id])

  return (start_seed + ''.join(text_generated))

In [ ]:
print(generate_text(model,"sweet",gen_size=1000))

sweet what, you see
  ISABELLA. If 'honest you bear me by his lip
    With all the caslicI let this bird!
    Since limbed as the poor dare you, I'll call it, give men,
    When I should swift-all despis'd? Come, our Kins aide,  
    What is it?
  PROTEUS. And thou the Emperor's brothers.  
    Mine honour, I assure you, on the rest;
    But, tom amongst them, and let us deliver you
        Young Caecame, let's best two friends.
    Acking that took upon the stemeland

      Enter ARIEL, o' th' ring, that is another foolish
    coward churched.
  BIONDELLO. Have you? What did you be much all durb'd in a face?
  Dear lady was I then put me for't. Prithee ask it: CESSS, and others
  HUBPORT and COLVILLES with our Agn;
    Lavely the waggon looke and all past grace.
  LUCIUS. Why, this is the sheep, and dull.
  PETRUCHIO. Come beaten, and I'll deliver you.
  IAGO. Nampony' sisters.
  MOTH. Ay, by the man.
  DUKE. Sir, I was never yet as any in the
    nephy marrow; if you will tell them h